### Instals

In [ ]:
import shutil
# Deletes 'CDTD-CFG' map if necessary
shutil.rmtree('/content/CDTD-CFG', ignore_errors=True)

In [ ]:
# Clone git and move into the repo
!git clone https://github.com/luethytommaso/CDTD-CFG.git
%cd CDTD-CFG/

In [ ]:
import sys
sys.path.append(".")

In [ ]:
!pip install requirements.txt
!pip install catboost
!pip install torch
!pip install sklearn
!pip install scikit learn
!pip install pandas
!pip install seaborn
!pip install torch_ema
#torch-ema

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import random

from cdtd_cfg import CDTD

# set all seeds
def set_seeds(seed, cuda_deterministic=False):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            if cuda_deterministic:
                print("cuda_deterministic = True")
                torch.backends.cudnn.deterministic = True
                torch.backends.cudnn.benchmark = False

set_seeds(42, cuda_deterministic=True)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from collections import Counter
from scipy.stats import gaussian_kde
from scipy.spatial.distance import jensenshannon
from scipy.stats import wasserstein_distance
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OrdinalEncoder, QuantileTransformer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder

"""
This code block contains all methods called outside of the CDTD and Synthpop frameworks throughout this analysis.
"""

def fit_cdtd_model(X_cat_train, X_cont_train, cfg, label_1_idx, label_2_idx, dropout_ratio=0.2, num_steps_train=3000, num_steps_warmup=1000):
    # Extract the target conditioning variables (two categorical labels) from the training data
    y_condition_1 = X_cat_train[:, label_1_idx]
    y_condition_2 = X_cat_train[:, label_2_idx]

    # Initialize the CDTD model with training data and conditioning labels
    cdtd = CDTD(
        X_cat_train = X_cat_train,
        X_cont_train = X_cont_train,
        cfg = cfg,
        y_condition_1=y_condition_1,
        y_condition_2=y_condition_2,
    )

    # Train the CDTD model with the given number of steps, warmup, and dropout
    cdtd.fit(X_cat_train, X_cont_train, num_steps_train=num_steps_train, num_steps_warmup=num_steps_warmup, cfg = cfg,
            y_condition_1=y_condition_1, y_condition_2=y_condition_2, dropout_ratio = dropout_ratio)

    return cdtd


def generate_samples(num_samples, cdtd_model, cat_int_enc, cont_enc, cont_std, cat_features, cont_features,
                     int_features, cfg, cfg_scale_1, cfg_scale_2, label_1, label_2, target_label_1, target_label_2,
                     X_cat_train, targeted_sampling=False, return_latents=False):

    # Locate indices of the two conditioning variables in the categorical features
    label_1_idx = cat_features.index(label_1)
    label_2_idx = cat_features.index(label_2)
    classes_label_1 = cat_int_enc.categories_[label_1_idx]
    classes_label_2 = cat_int_enc.categories_[label_2_idx]
    num_classes_1 = len(classes_label_1)
    num_classes_2 = len(classes_label_2)

    # Build probability matrix for sampling (ID vs OOD)
    if cfg: # CFG denotes whether we call CDTD model or CDTD-CFG model
        if targeted_sampling: # Targeted sampling denotes whether we call the CDTD-CFG model ID or OOD
            # Set only the probability of the OOD class pair to 1 if CFG-OOD
            target_label_1_idx = np.where(classes_label_1 == target_label_1)[0][0]
            target_label_2_idx = np.where(classes_label_2 == target_label_2)[0][0]
            probs_matrix = np.zeros((num_classes_1, num_classes_2))
            probs_matrix[target_label_1_idx, target_label_2_idx] = 1.0
        else:
            # Copy the joint class distribution from training data for ID sampling
            y_label_1_train = X_cat_train[:, label_1_idx].numpy()
            y_label_2_train = X_cat_train[:, label_2_idx].numpy()
            pair_counts = Counter(zip(y_label_1_train, y_label_2_train))
            total = sum(pair_counts.values())
            probs_matrix = np.zeros((num_classes_1, num_classes_2))
            for (l1, l2), count in pair_counts.items():
                probs_matrix[l1, l2] = count / total
    else:
        probs_matrix = None

    # Inverse-transform categorical and continuous features back to original data space
    X_cat_gen, X_cont_gen, latent_vectors = cdtd_model.sample(
        num_samples=num_samples,
        cfg=cfg,
        probs_matrix=probs_matrix,
        cfg_scale_1=cfg_scale_1,
        cfg_scale_2=cfg_scale_2,
        return_latents=return_latents
    )

    X_cat_gen_inv = cat_int_enc.inverse_transform(X_cat_gen)
    X_cont_gen_inv = cont_std.inverse_transform(X_cont_gen)
    X_cont_gen_inv = cont_enc.inverse_transform(X_cont_gen_inv)

    # Create a DataFrame with the correct column names and data types
    df_gen = pd.concat(
        (pd.DataFrame(X_cat_gen_inv), pd.DataFrame(X_cont_gen_inv)), axis=1
    )
    df_gen.columns = cat_features + cont_features
    df_gen[cont_features] = df_gen[cont_features].apply(pd.to_numeric, errors="coerce")
    df_gen[int_features] = df_gen[int_features].round().astype("int")
    return df_gen, latent_vectors


def plot_all_features(df_train, df_generated, cat_features, cont_features, cfg=False, targeted_sampling=False, removed_group_df=None, synthpop_ood_df = None):
    # Plot distributions of all categorical and continuous features
    # Compares train data vs generated (CDTD-CFG & synthpop OOD) vs removed data

    n_total = len(cat_features) + len(cont_features)
    n_cols = 3
    n_rows = (n_total + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 3 * n_rows))
    axes = axes.flatten()

    for i, label in enumerate(cat_features):
        _plot_cat(df_train, df_generated, label, axes[i], cfg, targeted_sampling, removed_group_df, synthpop_ood_df)
    for j, label in enumerate(cont_features):
        _plot_cont(df_train, df_generated, label, axes[len(cat_features) + j], cfg, targeted_sampling, removed_group_df, synthpop_ood_df)

    for ax in axes:
        ax.legend()
    plt.tight_layout()
    plt.show()

def _plot_cat(df_train, df_generated, label, ax, cfg, targeted_sampling, removed_group_df, synthpop_ood_df):
    # Plots categorical feature distribution as bar charts.
    # Aligns category labels across train, generated, removed, and synthpop sets.
    df_train[label] = df_train[label].astype(str)
    df_generated[label] = df_generated[label].astype(str)
    if removed_group_df is not None:
        removed_group_df = removed_group_df.copy()
        removed_group_df[label] = removed_group_df[label].astype(str)
    if synthpop_ood_df is not None:
        synthpop_ood_df[label] = synthpop_ood_df[label].astype(str)

    props_train = df_train[label].value_counts(normalize=True)
    props_gen = df_generated[label].value_counts(normalize=True)
    props_removed = removed_group_df[label].value_counts(normalize=True) if (cfg and targeted_sampling and removed_group_df is not None) else pd.Series(dtype=float)
    props_synth = synthpop_ood_df[label].value_counts(normalize=True) if (cfg and targeted_sampling and synthpop_ood_df is not None) else pd.Series(dtype=float)

    all_cats = sorted(set(props_removed.index).union(props_train.index).union(props_gen.index).union(props_synth.index))
    x = np.arange(len(all_cats))
    y_removed = [props_removed.get(cat, 0.0) for cat in all_cats]
    y_train = [props_train.get(cat, 0.0) for cat in all_cats]
    y_gen = [props_gen.get(cat, 0.0) for cat in all_cats]
    y_synthpop = [props_synth.get(cat, 0.0) for cat in all_cats]

    n_bars = 4 if cfg and targeted_sampling else 2
    bar_width = 0.8 / n_bars
    offset = [-1.5, -0.5, 0.5, 1.5] if n_bars == 4 else [-0.15, 0.15]
    if cfg and targeted_sampling:
        ax.bar(x + offset[2]*bar_width, y_removed, width=bar_width, label="Removed Data", color="tab:orange")
        ax.bar(x + offset[3]*bar_width, y_synthpop, width=bar_width, label="Synthpop OOD", color="tab:purple")
    ax.bar(x + offset[0]*bar_width, y_train, width=bar_width, label="Train Only Data", color="tab:green")
    ax.bar(x + offset[1]*bar_width, y_gen, width=bar_width, label="CDTD-OOD", color="tab:blue")
    ax.set_title(label)
    ax.set_xticks(x)
    ax.set_xticklabels(all_cats, rotation=60, fontsize=7)


def _plot_cont(df_train, df_generated, label, ax, cfg, targeted_sampling, removed_group_df, synthpop_ood_df):
    # Plot continuous feature distribution using KDE curves.
    if cfg and targeted_sampling and removed_group_df is not None and len(removed_group_df) > 0:
        sns.kdeplot(removed_group_df[label], ax=ax, label="Removed Data", color="tab:orange")
    if cfg and targeted_sampling and synthpop_ood_df is not None and len(synthpop_ood_df) > 0:
        sns.kdeplot(synthpop_ood_df[label], ax=ax, label="Synthpop OOD", color="tab:purple")
    sns.kdeplot(df_train[label], ax=ax, label="Train Only Data", color="tab:green")
    sns.kdeplot(df_generated[label], ax=ax, label="CDTD-OOD", color="tab:blue")
    ax.set_title(label)

def create_train_test_combinations_ext(
    X_cat_train,
    X_cont_train,
    X_cat_test,
    X_cont_test,
    df_generated_ID=None,
    df_generated_OOD=None,
    df_generated_synthpop_ID=None,
    df_generated_synthpop_OOD=None,
    removed_group_df=None,
    cat_features=None,
    cont_features=None,
    label_1=None,
    label_2=None,
    target_label_1=None,
    target_label_2=None,
    classification_variable = None,
    classification_target = None,
    cat_int_enc=None,
    OOD_share=None
):
    """
    Build all training dataset combinations:
    - train only, +ID, +OOD, +both, + real, and train combined baseline
    Decodes encoded train/test sets, adds target column, and filters OOD to target subgroup.
    Splits removed data, constructs df_test, and ensures feature type consistency.
    Returns dictionary of all train sets, the cleaned test set, and the categorical feature list that is contained in the X set.
    """

    def add_target(df):
        df = df.copy()
        df["target"] = (df[classification_variable] == classification_target).astype(int)
        return df

    # Decode train and test data back into data space form and transform into daataframes with original feature names
    X_cat_train_decoded = cat_int_enc.inverse_transform(X_cat_train.numpy())
    X_cat_test_decoded = cat_int_enc.inverse_transform(X_cat_test.numpy())
    X_cont_train_real = cont_enc.inverse_transform(cont_std.inverse_transform(X_cont_train.numpy()))
    X_cont_test_real = cont_enc.inverse_transform(cont_std.inverse_transform(X_cont_test.numpy()))

    df_train_raw = pd.DataFrame(np.concatenate([X_cat_train_decoded, X_cont_train_real], axis=1),
                                columns=cat_features + cont_features)
    df_test_raw = pd.DataFrame(np.concatenate([X_cat_test_decoded, X_cont_test_real], axis=1),
                               columns=cat_features + cont_features)

    df_train_raw[cont_features] = df_train_raw[cont_features].apply(pd.to_numeric, errors="coerce")
    df_test_raw[cont_features] = df_test_raw[cont_features].apply(pd.to_numeric, errors="coerce")
    df_train_raw = add_target(df_train_raw)
    df_test_raw = add_target(df_test_raw)

    # Split the removed group over the test set and a part added to the train + real data set
    if removed_group_df is not None:
        removed_group_df_half_1, removed_group_df_half_2 = train_test_split(
            removed_group_df, test_size=(1 - test_split), random_state=42
        )
        removed_group_df_half_1 = add_target(removed_group_df_half_1)
        removed_group_df_half_2 = add_target(removed_group_df_half_2)
        df_test = pd.concat([df_test_raw, removed_group_df_half_1], axis=0).reset_index(drop=True)
    else:
        df_test = df_test_raw.copy()
        removed_group_df_half_2 = None

    # Add target to synthetic sets
    for df in [df_generated_ID, df_generated_OOD, df_generated_synthpop_ID, df_generated_synthpop_OOD]:
        if df is not None:
            df["target"] = (df[classification_variable] == classification_target).astype(int)

    # Filter synthetic OOD rows to contain the deleted subgroup only
    df_generated_OOD_filtered = df_generated_OOD[
        (df_generated_OOD[label_1] == target_label_1) & (df_generated_OOD[label_2] == target_label_2)
    ].copy() if df_generated_OOD is not None else None

    df_generated_synthpop_OOD_filtered = df_generated_synthpop_OOD[ # should remove no data, since synthpop exclusively contains target group data
        (df_generated_synthpop_OOD[label_1] == target_label_1) & (df_generated_synthpop_OOD[label_2] == target_label_2)
    ].copy() if df_generated_synthpop_OOD is not None else None

    # Ensure consistent types
    for df in [df_generated_ID, df_generated_OOD_filtered,
               df_generated_synthpop_ID, df_generated_synthpop_OOD_filtered,
               removed_group_df]:
        if df is not None:
            for col in cont_features:
                df[col] = pd.to_numeric(df[col], errors="coerce")

    # Build all combinations
    train_combinations = {
        "only_train": df_train_raw,
        "train_plus_id": pd.concat([df_train_raw, df_generated_ID], axis=0) if df_generated_ID is not None else None,
        "train_plus_ood": pd.concat([df_train_raw, df_generated_OOD_filtered.iloc[:int(len(df_train_raw) * OOD_share)]], axis=0) if df_generated_OOD_filtered is not None else None,
        "train_plus_ood_small": pd.concat([df_train_raw, df_generated_OOD_filtered.iloc[:int(len(df_train_raw) * OOD_share * 0.5)]], axis=0) if df_generated_OOD_filtered is not None else None,
        "train_plus_ood_large": pd.concat([df_train_raw, df_generated_OOD_filtered.iloc[:int(len(df_train_raw) * OOD_share * 1.5)]], axis=0) if df_generated_OOD_filtered is not None else None,
        "train_plus_both": pd.concat([df_train_raw, df_generated_ID, df_generated_OOD_filtered.iloc[:int((len(df_train_raw)+len(df_generated_ID)) * OOD_share)]], axis=0)
            if df_generated_ID is not None and df_generated_OOD_filtered is not None else None,
        "train_plus_synthpop_id": pd.concat([df_train_raw, df_generated_synthpop_ID], axis=0)
            if df_generated_synthpop_ID is not None else None,
        "train_plus_synthpop_ood": pd.concat([df_train_raw, df_generated_synthpop_OOD_filtered.iloc[:int(len(df_train_raw) * OOD_share)]], axis=0)
            if df_generated_synthpop_OOD_filtered is not None else None,
        "train_plus_synthpop_both": pd.concat([df_train_raw, df_generated_synthpop_ID, df_generated_synthpop_OOD_filtered.iloc[:int((len(df_train_raw)+len(df_generated_synthpop_ID)) * OOD_share)]], axis=0)
            if df_generated_synthpop_ID is not None and df_generated_synthpop_OOD_filtered is not None else None,
        "train_plus_real": pd.concat([df_train_raw, removed_group_df_half_2], axis=0),
        "train_combined": df_train_raw,
    }

    # Finalize test set by splitting into X and y
    X_test = df_test.drop(columns=["target", classification_variable])
    y_test = df_test["target"]

    # Remove classification variable from feature list (since it is now contained in y variable).
    cat_features_cleaned = [col for col in cat_features if col != classification_variable]

    for col in cat_features_cleaned:
        X_test[col] = X_test[col].astype(str)

    return train_combinations, X_test, y_test, cat_features_cleaned, df_test

def train_classifier(X_train, y_train, cat_features, cont_features, model, model_name):
    # Train the classifiers using a preprocessing pipeline.
    # Encodes categorical features (OneHot for Logistic Regression, Ordinal otherwise) and scales continuous features and fits the given model.

    for col in cat_features:
        X_train[col] = X_train[col].astype(str)

    if model_name == "LogisticRegression":
        preprocessor = ColumnTransformer([
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
            ("cont", StandardScaler(), cont_features),
        ])

    else:
        preprocessor = ColumnTransformer([
            ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), cat_features),
            ("cont", StandardScaler(), cont_features),
        ])

    clf_pipeline = Pipeline([
        ("preprocess", preprocessor),
        ("clf", model)
    ])
    clf_pipeline.fit(X_train, y_train)
    return clf_pipeline

def evaluate_classifier(clf_pipeline, X_test, y_test, cat_features):
    # Evaluate trained classifier on test data.
    # Allows for printing of classification report and confusion matrix for performance insights.
    for col in cat_features:
        X_test[col] = X_test[col].astype(str)

    y_pred = clf_pipeline.predict(X_test)

    print("=== Classifier Report ===")
    print(classification_report(y_test, y_pred))

    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(cm,
                         index=["Actual 0", "Actual 1"],
                         columns=["Predicted 0", "Predicted 1"])
    print("\n=== Confusion Matrix ===")
    print(cm_df)

def generate_samples_synthpop(
    synthpop_model,
    num_samples,
    X_cat_train,
    cat_int_enc,
    label_1,
    label_2,
    target_label_1,
    target_label_2,
    targeted_sampling=True
):
    # Generate samples using a Synthpop model. Supports targeted OOD sampling (only passing target labels) or ID sampling (copying train distribution).

    if targeted_sampling:
        # Get label indices and category mappings
        label_1_idx = synthpop_model.x_cat_indices[0]
        label_2_idx = synthpop_model.x_cat_indices[1]
        classes_label_1 = cat_int_enc.categories_[label_1_idx]
        classes_label_2 = cat_int_enc.categories_[label_2_idx]

        # Find target label indices
        target_label_1_idx = np.where(classes_label_1 == target_label_1)[0][0]
        target_label_2_idx = np.where(classes_label_2 == target_label_2)[0][0]

        # X_fixed_cat is set to target values for all samples for OOD group
        X_fixed_cat = np.full((num_samples, 2), fill_value=[target_label_1_idx, target_label_2_idx], dtype=int)
        X_fixed_cont = None

    else:
        # ID sampling we copy the training distribution
        X_fixed_cat = X_cat_train[:, synthpop_model.x_cat_indices].numpy()
        X_fixed_cont = None

    # Sample from Synthpop
    x_cat_gen, x_cont_gen = synthpop_model.sample(X_fixed_cat, X_fixed_cont)

    # Decode into full DataFrame
    df_generated = decode_generated_samples(x_cat_gen, x_cont_gen, cat_int_enc, label_1, label_2)
    return df_generated

def decode_generated_samples(x_cat_gen, x_cont_gen, cat_int_enc, label_1, label_2):
    # Decode raw Synthpop categorical and continuous outputs into a DataFrame.
    # Places generated values in the correct feature positions and inverse-transforms categories and adds continuous columns if present.

    x_cat_full = np.zeros((x_cat_gen.shape[0], len(cat_int_enc.feature_names_in_)), dtype=int)
    label_1_idx = list(cat_int_enc.feature_names_in_).index(label_1)
    label_2_idx = list(cat_int_enc.feature_names_in_).index(label_2)
    x_cat_full[:, label_1_idx] = x_cat_gen[:, 0]
    x_cat_full[:, label_2_idx] = x_cat_gen[:, 1]

    decoded = cat_int_enc.inverse_transform(x_cat_full)
    df_cats = pd.DataFrame(decoded, columns=cat_int_enc.feature_names_in_)

    if x_cont_gen is not None:
        # generic names are given to continuous columns to distinguish them from categorical columns
        df_conts = pd.DataFrame(x_cont_gen, columns=[f"cont_{i}" for i in range(x_cont_gen.shape[1])])
        df = pd.concat([df_cats, df_conts], axis=1)
    else:
        df = df_cats

    return df

def decode_synthpop_samples(
    x_cat_gen,
    x_cont_gen,
    X_fixed_cat,
    label_1_idx,
    label_2_idx,
    cat_features,
    cont_features,
    int_features,
    cat_int_enc,
    cont_enc,
    cont_std
):
    # Rebuilds full sample DataFrame from Synthpop outputs and fixed conditioning variables.
    # Restores categorical variables via encoder, inverse-scales continuous values, and rounds ints. Ensures output matches original feature order.

    x_cat_full = np.empty((x_cat_gen.shape[0], len(cat_features)), dtype=int)

    for i, idx in enumerate([label_1_idx, label_2_idx]):
        x_cat_full[:, idx] = X_fixed_cat[:, i]

    gen_indices = [i for i in range(len(cat_features)) if i not in [label_1_idx, label_2_idx]]
    for i, idx in enumerate(gen_indices):
        x_cat_full[:, idx] = x_cat_gen[:, i]

    x_cat_gen_inv = cat_int_enc.inverse_transform(x_cat_full)
    df_cats = pd.DataFrame(x_cat_gen_inv, columns=cat_features)
    x_cont_inv = cont_std.inverse_transform(x_cont_gen)
    x_cont_inv = cont_enc.inverse_transform(x_cont_inv)
    df_cont = pd.DataFrame(x_cont_inv, columns=cont_features)

    df_out = pd.concat([df_cats, df_cont], axis=1)
    df_out[int_features] = df_out[int_features].round().astype(int)

    return df_out

def summarize_target_overlap(df, label_1, label_2, target_label_1, target_label_2, name="Dataset"):
    # Print summary of how many rows in a dataset match one or both target labels. Helps understand the distribution of (synthetic) data.
    print(f"\n=== {name} ===")
    cond_1 = df[label_1] == target_label_1
    cond_2 = df[label_2] == target_label_2
    both_match = (cond_1 & cond_2).sum()
    only_1 = (cond_1 & ~cond_2).sum()
    only_2 = (~cond_1 & cond_2).sum()
    neither = (~cond_1 & ~cond_2).sum()
    total = len(df)
    print(f"Total rows: {total}")
    print(f"Match both [{target_label_1}, {target_label_2}]: {both_match} ({both_match/total:.2%})")
    print(f"Only {label_1} == {target_label_1}: {only_1} ({only_1/total:.2%})")
    print(f"Only {label_2} == {target_label_2}: {only_2} ({only_2/total:.2%})")
    print(f"Match neither: {neither} ({neither/total:.2%})")

def compute_distance_metrics(real_df, synthetic_df, cat_features, cont_features):
    # Compare real vs. synthetic datasets using JSD for categorical distributions and Wasserstein distance for continuous variables
    # Also checks and reports datatype mismatches between datasets.

    type_mismatches = {}
    for feature in cat_features + cont_features:
        real_dtype = real_df[feature].dtype
        synth_dtype = synthetic_df[feature].dtype
        if real_dtype != synth_dtype:
            type_mismatches[feature] = (real_dtype, synth_dtype)

    jsd_results = {}
    wsd_results = {}

    for feature in cat_features:
        real_counts = real_df[feature].astype(str).value_counts(normalize=True)
        synth_counts = synthetic_df[feature].astype(str).value_counts(normalize=True)

        all_cats = list(set(real_counts.index) | set(synth_counts.index))
        real_probs = np.array([real_counts.get(cat, 0) for cat in all_cats])
        synth_probs = np.array([synth_counts.get(cat, 0) for cat in all_cats])

        real_probs += 1e-12
        synth_probs += 1e-12

        jsd = jensenshannon(real_probs, synth_probs, base=2)
        jsd_results[feature] = jsd

    for feature in cont_features:
        real_values = pd.to_numeric(real_df[feature], errors="coerce").dropna()
        synth_values = pd.to_numeric(synthetic_df[feature], errors="coerce").dropna()
        wsd = wasserstein_distance(real_values, synth_values)
        wsd_results[feature] = wsd

    result_summary = {
        "JSD_mean": np.mean(list(jsd_results.values())) if jsd_results else None,
        "WSD_mean": np.mean(list(wsd_results.values())) if wsd_results else None,
    }

    return result_summary


### Dataset preprocessing and hyperparameter initialization

In [ ]:
#Setup 1, missing class combination = Circulatory + African American
label_1 = "diagnosis_group"
target_label_1 = "Circulatory"

# Setup 2, missing class combination = readmitted + African American
# label_1 = "readmitted"
# target_label_1 = "yes"

target_label_2 = "AfricanAmerican"
label_2 = "race"

# I none, label 1 is the classification variable (used in subgroup related classification)
classification_target = None
classification_variable = None
# Can be used in combination with Setup 1 to get the subgroup unrelated classification results
# classification_target = "yes"
# classification_variable = "readmitted"

# Hyperparameters used in all scenarios throughout the paper under these exact settings.
sample_size_scale = 1 # Multiplied by the size of the train data in order to decide how large the synthetic sample will be
dropout_ratio = 0.2 # Dropout ratio deciding what part of training is performed unconditionally
num_steps_train = 30000 # Number of training steps of the CDTD-CFG MLP
num_steps_warmup = 1000 # Number of warmup steps of the CDTD-CFG MLP
min_nonmissing_ratio = 0.9 # States that no more than 10% of all categories can be None before being dropped
test_split = 0.2 # Ratio of data used for the test set.
synthpop_tree_depth = 10 # Max tree depth in synthpop applications
min_freq = 0.001 # Minimum frequency of a category before being merged with Other
OOD_scale = 1 # Size of the OOD samples we want to create with respects to the training sample size

In [ ]:
### Load and clean data

df = pd.read_csv("data/diabetic_data.csv")

# Keep only the first encounter per patient, drop ID's and shuffle dataset
df = df.sort_values("encounter_id").drop_duplicates("patient_nbr", keep="first")
df.drop(columns=["encounter_id", "patient_nbr"], inplace=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Empty strings, spaces, None, and emtpy are set to NaN
df = df.replace(r"^\s*$", np.nan, regex=True)
df = df.replace("?", np.nan)
df = df.replace(
    to_replace=r"(?i)^\s*(none|empty|\?)\s*$",
    value=np.nan,
    regex=True
)

# Creating count variable of number of diabetes medications taken (excluding insulin), and subsequently drop them
diabetes_meds = [
    "metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride",
    "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone",
    "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide",
    "examide", "citoglipton", "glyburide-metformin", "glipizide-metformin",
    "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"
]
df["num_diabetes_meds"] = df[diabetes_meds].apply(
    lambda row: sum(val in ["Up", "Down", "Steady"] for val in row), axis=1
)
df.drop(columns=diabetes_meds, inplace=True)

# Diagnosis group mapping, to replace three distinct diagnosis by a single group
def map_diagnosis_group(code):
    try:
        code = float(code)
    except:
        return "Other"

    if (390 <= code <= 459) or (code == 785):
        return "Circulatory"
    elif (460 <= code <= 519) or (code == 786):
        return "Respiratory"
    elif (520 <= code <= 579) or (code == 787):
        return "Digestive"
    elif code == 250:
        return "Diabetes"
    elif (800 <= code <= 999):
        return "Injury"
    elif (710 <= code <= 739):
        return "Musculoskeletal"
    elif (580 <= code <= 629) or (code == 788):
        return "Genitourinary"
    elif (140 <= code <= 239):
        return "Neoplasms"
    else:
        return "Other"

df["diagnosis_group"] = df["diag_1"].apply(map_diagnosis_group)
df.drop(columns=["diag_1", "diag_2", "diag_3"], inplace=True)


# Ethnicity mapping to create a larger Other set
def map_etnicity(ethnicity):
    if ethnicity in ["?", "Asian", "Hispanic", "Other"]:
        return "Other"
    else:
        return ethnicity
df["race"] = df["race"].apply(map_etnicity)


# Transform readmitted to boolean, by settings al readmission variables to yes
df["readmitted"] = df["readmitted"].apply(lambda x: "yes" if x == "<30" or x == ">30" else "no")

# Define categorical, continuous, and integer feature lists
cat_features = [
    "race", "gender", "age", "admission_type_id", "discharge_disposition_id",
    "payer_code", "medical_specialty", "max_glu_serum", "A1Cresult",
    "insulin", "change", "diabetesMed", "readmitted", "diagnosis_group"
]
cont_features = [col for col in df.columns if col not in cat_features]
int_features = [
    "admission_source_id", "time_in_hospital", "num_lab_procedures",
    "num_procedures", "num_medications", "number_outpatient",
    "number_emergency", "number_inpatient", "number_diagnoses",
    "num_diabetes_meds"
]

# Drop rows with missing diagnosis group and features with too many missing categories
df = df[df["diagnosis_group"].notna()]
df = df.loc[:, df.notna().mean() >= min_nonmissing_ratio]

# Update feature lists to only contain remaining columns
cat_features = [col for col in cat_features if col in df.columns]
cont_features = [col for col in cont_features if col in df.columns]
int_features = [col for col in int_features if col in df.columns]

# Now fill remaining categorical NaNs with string 'empty' and drop continuous rows with missing values
df[cat_features] = df[cat_features].fillna("empty")
df.dropna(inplace=True)

for col in cat_features:
    freqs = df[col].value_counts(normalize=True)
    rare = freqs[freqs < min_freq].index
    df[col] = df[col].apply(lambda x: "other" if x in rare else x)

# Remove the target group from the training set dataset, storing it in a different dataset for later use
# We compute the OOD_target (how prevalent was the group in the original dataset), and compute the OOD_Share we need to add on top of the training data to match this target.
removed_group_df = df[(df[label_1] == target_label_1) & (df[label_2] == target_label_2)].copy()
OOD_target = removed_group_df.shape[0] / df.shape[0]
OOD_share_df = OOD_target/(1 - OOD_target)
df = df[~((df[label_1] == target_label_1) & (df[label_2] == target_label_2))]

# ensure correct types
X_cat = df[cat_features].to_numpy().astype("str")
X_cont = df[cont_features].to_numpy().astype("float")

# Encode categorical and continuous data
X_cat_df = df[cat_features].astype(str)
cat_int_enc = OrdinalEncoder()
X_cat_enc = cat_int_enc.fit_transform(X_cat_df)

X_cont = df[cont_features].astype(float).to_numpy()
cont_enc = QuantileTransformer(
    output_distribution="normal",
    n_quantiles=max(min(X_cont.shape[0] // 30, 1000), 10),
    subsample=int(1e9),
    random_state=42,
)

X_cont_enc = cont_enc.fit_transform(X_cont)
cont_std = StandardScaler()
X_cont_enc_std = cont_std.fit_transform(X_cont_enc)

# Convert to PyTorch tensors and split into train and test set
X_cat_tensor = torch.tensor(X_cat_enc).long()
X_cont_tensor = torch.tensor(X_cont_enc_std).float()

X_cat_train, X_cat_test, X_cont_train, X_cont_test = train_test_split(
    X_cat_tensor, X_cont_tensor, test_size=test_split, random_state=42, shuffle=True
)

X_cat_train = torch.tensor(X_cat_train).long()
X_cat_test = torch.tensor(X_cat_test).long()
X_cont_train = torch.tensor(X_cont_train).float()
X_cont_test = torch.tensor(X_cont_test).float()


# Sets hyperparams used in the training and sampling process
num_samples = round(X_cat_train.shape[0]) * sample_size_scale
label_1_idx = cat_features.index(label_1)
label_2_idx = cat_features.index(label_2)
OOD_share = OOD_share_df * OOD_scale

### CDTD-CFG

In [ ]:
# Train the CDTD-CFG model
cdtd_cfg_model = fit_cdtd_model(
    X_cat_train=X_cat_train,
    X_cont_train=X_cont_train,
    cfg=True,
    label_1_idx=label_1_idx,
    label_2_idx=label_2_idx,
    dropout_ratio=dropout_ratio,
    num_steps_train=num_steps_train,
    num_steps_warmup=num_steps_warmup
)

# Not used within the final thesis, but shows a CDTD-CFG ID sample call
# df_CFG_generated_ID = generate_samples(
#     num_samples=round(num_samples),
#     cat_int_enc=cat_int_enc,
#     cont_enc=cont_enc,
#     cont_std=cont_std,
#     cat_features=cat_features,
#     cont_features=cont_features,
#     int_features=int_features,
#     cfg=cfg,
#     cfg_scale_1=0.4,
#     cfg_scale_2=0.4,
#     targeted_sampling=False,
#     target_label_1=target_label_1,
#     target_label_2=target_label_2,
#     X_cat_train=X_cat_train,
#     label_1=label_1,
#     label_2=label_2,
#     cdtd_model=cdtd_cfg_model
# )

# Define guidance weight grid search, saving the combination that yields the best match % for further analysis
cfg_scales = np.linspace(0.3, 0.7, 5)
match_results = []
best_match_percentage = -1
df_generated_OOD = None
best_cfg_scales = (None, None)

# Loop over all combinations
for scale_1 in cfg_scales:
    for scale_2 in cfg_scales:
        print("scale_1: ", scale_1, " scale_2: ", scale_2)
        df_generated_OOD_draw, latent_vectors = generate_samples(
            num_samples=round(num_samples),
            cat_int_enc=cat_int_enc,
            cont_enc=cont_enc,
            cont_std=cont_std,
            cat_features=cat_features,
            cont_features=cont_features,
            int_features=int_features,
            cfg=True,
            cfg_scale_1=scale_1,
            cfg_scale_2=scale_2,
            targeted_sampling=True,
            target_label_1=target_label_1,
            target_label_2=target_label_2,
            X_cat_train=X_cat_train,
            label_1=label_1,
            label_2=label_2,
            cdtd_model=cdtd_cfg_model,
            return_latents= False # Can be used for additional analyses, to see how the latent sample affects the data quality
        )

        # Compute match % and distance metrics
        match_mask = (df_generated_OOD_draw[label_1] == target_label_1) & (df_generated_OOD_draw[label_2] == target_label_2)
        match_percentage = match_mask.mean() * 100

        df_generated_OOD_filtered_draw = df_generated_OOD_draw[
            (df_generated_OOD_draw[label_1] == target_label_1) & (df_generated_OOD_draw[label_2] == target_label_2)
        ]
        metrics_cdtd_draw = compute_distance_metrics(removed_group_df, df_generated_OOD_filtered_draw, cat_features, cont_features)
        print(metrics_cdtd_draw)

        match_results.append({
            "cfg_scale_1": scale_1,
            "cfg_scale_2": scale_2,
            "match_percentage": match_percentage,
            "JSD_mean": metrics_cdtd_draw["JSD_mean"],
            "WSD_mean": metrics_cdtd_draw["WSD_mean"]
        })

        if match_percentage > best_match_percentage:
            best_match_percentage = match_percentage
            df_generated_OOD = df_generated_OOD_draw.copy()
            best_cfg_scales = (scale_1, scale_2)

match_df = pd.DataFrame(match_results)

match_table = match_df.pivot(index="cfg_scale_1", columns="cfg_scale_2", values="match_percentage")
print("\n=== Match % for each cfg_scale combination ===\n")
print(match_table.round(2))

jsd_table = match_df.pivot(index="cfg_scale_1", columns="cfg_scale_2", values="JSD_mean")
print("\n=== JSD_mean for each cfg_scale combination ===\n")
print(jsd_table.round(4))

wsd_table = match_df.pivot(index="cfg_scale_1", columns="cfg_scale_2", values="WSD_mean")
print("\n=== WSD_mean for each cfg_scale combination ===\n")
print(wsd_table.round(4))

print(best_cfg_scales,best_match_percentage)


### CDTD-ID

In [ ]:
# Train the CDTD model
cdtd_model = fit_cdtd_model(
    X_cat_train=X_cat_train,
    X_cont_train=X_cont_train,
    cfg=False,
    label_1_idx=label_1_idx,
    label_2_idx=label_2_idx,
    dropout_ratio=dropout_ratio,
    num_steps_train=num_steps_train,
    num_steps_warmup=num_steps_warmup
)

# Generate CDTD ID samples
df_generated_ID, latent_vectors = generate_samples(
    num_samples=round(num_samples),
    cat_int_enc=cat_int_enc,
    cont_enc=cont_enc,
    cont_std=cont_std,
    cat_features=cat_features,
    cont_features=cont_features,
    int_features=int_features,
    cfg=False,
    cfg_scale_1=0,
    cfg_scale_2=0,
    targeted_sampling=False,
    target_label_1=target_label_1,
    target_label_2=target_label_2,
    X_cat_train=X_cat_train,
    label_1=label_1,
    label_2=label_2,
    cdtd_model=cdtd_model,
    return_latents=False
)


### SYNTHPOP


In [ ]:
from synthpop import Synthpop

# Indices of the conditioning variables
x_cat_indices = [label_1_idx, label_2_idx]

# Create and fit the synthpop model (both ID and OOD trees at once)
synthpop_model = Synthpop(
    cat_train=X_cat_train,
    cont_train=X_cont_train,
    x_cat_indices=x_cat_indices,
    cfg=True,
    max_depth=10
)

_ = synthpop_model.fit()


In [ ]:
# for ID sampling we copy the training conditioning as input for the two target categories
X_fixed_cat_ID = X_cat_train[:, x_cat_indices].numpy()
X_fixed_cont_ID = None

x_cat_gen_ID, x_cont_gen_ID, X_fixed_cat_ID, X_fixed_cont_ID = synthpop_model.sample(
    X_fixed_cat=X_fixed_cat_ID,
    X_fixed_cont=X_fixed_cont_ID,
    targeted_sampling=False
)

# For OOD generation, we build a conditioning matrix where verey row has the same input of the target combinations
label_1_idx = cat_features.index(label_1)
label_2_idx = cat_features.index(label_2)
classes_label_1 = cat_int_enc.categories_[label_1_idx]
classes_label_2 = cat_int_enc.categories_[label_2_idx]

target_label_1_idx = np.where(classes_label_1 == target_label_1)[0][0]
target_label_2_idx = np.where(classes_label_2 == target_label_2)[0][0]
X_fixed_cat_OOD = np.full((round(num_samples), 2), fill_value=[target_label_1_idx, target_label_2_idx], dtype=int)
X_fixed_cont_OOD = None

x_cat_gen_OOD, x_cont_gen_OOD, X_fixed_cat_OOD, X_fixed_cont_OOD = synthpop_model.sample(
    X_fixed_cat=X_fixed_cat_OOD,
    X_fixed_cont=X_fixed_cont_OOD,
    targeted_sampling=True
)

# Sampling OOD and ID synthpop data
df_generated_synthpop_OOD = decode_synthpop_samples(
    x_cat_gen=x_cat_gen_OOD,
    x_cont_gen=x_cont_gen_OOD,
    X_fixed_cat=X_fixed_cat_OOD,
    label_1_idx=label_1_idx,
    label_2_idx=label_2_idx,
    cat_features=cat_features,
    cont_features=cont_features,
    int_features=int_features,
    cat_int_enc=cat_int_enc,
    cont_enc=cont_enc,
    cont_std=cont_std
)

df_generated_synthpop_ID = decode_synthpop_samples(
    x_cat_gen=x_cat_gen_ID,
    x_cont_gen=x_cont_gen_ID,
    X_fixed_cat=X_fixed_cat_ID,
    label_1_idx=label_1_idx,
    label_2_idx=label_2_idx,
    cat_features=cat_features,
    cont_features=cont_features,
    int_features=int_features,
    cat_int_enc=cat_int_enc,
    cont_enc=cont_enc,
    cont_std=cont_std
)


### Visualize and compare

In [ ]:
df_generated_OOD_filtered = df_generated_OOD[
    (df_generated_OOD[label_1] == target_label_1) & (df_generated_OOD[label_2] == target_label_2)
].copy() if df_generated_OOD is not None else None

# Visualization of all feature distributions
plot_all_features(df, df_generated_OOD_filtered, cat_features, cont_features, cfg=True, targeted_sampling=True, removed_group_df=removed_group_df, synthpop_ood_df=df_generated_synthpop_OOD)

# By uncommeting and re-running in Setup 1, we get the for subgroup-untrelated output
# classification_target = "yes"
# classification_variable = "readmitted"

# If no distinct classification target, we classify on variable label_1 (Disease = Circulatory, or readmitted = yes)
if classification_target == None and classification_variable == None:
    classification_variable = label_1
    classification_target = target_label_1

# Creating all augmentation setups
train_combinations, X_test, y_test, cat_features_cleaned, df_test = create_train_test_combinations_ext(
    X_cat_train,
    X_cont_train,
    X_cat_test,
    X_cont_test,
    df_generated_ID=df_generated_ID,
    df_generated_OOD=df_generated_OOD,
    df_generated_synthpop_ID=df_generated_synthpop_ID,
    df_generated_synthpop_OOD=df_generated_synthpop_OOD,
    removed_group_df=removed_group_df,
    cat_features=cat_features,
    cont_features=cont_features,
    label_1=label_1,
    label_2=label_2,
    target_label_1=target_label_1,
    target_label_2=target_label_2,
    classification_variable = classification_variable,
    classification_target = classification_target,
    cat_int_enc=cat_int_enc,
    OOD_share=OOD_share
)

# Initializing all classifiers
models_to_evaluate = {
    "XGBoost": XGBClassifier(max_depth=3, use_label_encoder=False, eval_metric="logloss", random_state=42, verbosity = 0),
    "LogisticRegression": LogisticRegression(
            max_iter=1000,
            random_state=42
        ),
    "RandomForest": RandomForestClassifier(
            n_estimators=100,
            max_depth=12,
            random_state=42
        ),
    }

results = []
histogram_data = {}

# For every augmentation strategy, we train and test the classifiers
for dataset_name, df_train in train_combinations.items():
    if df_train is None:
        continue

    target_label_2_copy = target_label_2

    # For the combined dataset, transform the African American Class into other for both train and test data
    if dataset_name == "train_combined":
        target_label_2_copy = "Other"
        df_train[label_2] = df_train[label_2].replace(target_label_2, target_label_2_copy)
        df_test[label_2] = df_test[label_2].replace(target_label_2, target_label_2_copy).copy()
        X_test[label_2] = X_test[label_2].replace(target_label_2, target_label_2_copy).copy()


    print(f"\n=================== Dataset: {dataset_name} ===================")

    # Summarize the overlap between target subgroups in the training set
    summarize_target_overlap(
        df_train,
        label_1=label_1,
        label_2=label_2,
        target_label_1=target_label_1,
        target_label_2=target_label_2_copy,
        name=dataset_name
    )

    # Split features for training and testing, and ensure all variables have the correct data type
    X_train = df_train.drop(columns=["target", classification_variable])
    y_train = df_train["target"]
    X_test_array = X_test.copy()
    y_test_array = y_test.copy()

    for col in cat_features_cleaned:
        X_train[col] = X_train[col].astype(str)
        X_test_array[col] = X_test_array[col].astype(str)
    for col in int_features:
        X_train[col] = X_train[col].astype(int)
        X_test_array[col] = X_test_array[col].astype(int)

    # Train and evaluate each classifier
    for model_name, model in models_to_evaluate.items():

        # Fit classifier and predict labels and probabilities on the test set
        clf = train_classifier(X_train.copy(), y_train.copy(), cat_features_cleaned, cont_features, model, model_name)
        y_pred = clf.predict(X_test_array)
        y_proba = clf.predict_proba(X_test_array)[:, 1]

        # Mask for African American subgroup
        mask_target = (df_test[label_2] == target_label_2_copy).to_numpy()

        # For train only and train plus OOD print the feature importances (reported in results)
        if dataset_name in ["only_train", "train_plus_ood"]:
            print(f"\nRace importance for {model_name} ({dataset_name}):")

            if model_name == "XGBoost":
                print("Race importance:", model.feature_importances_[0])

            elif model_name == "RandomForest":
                print("Race importance:", model.feature_importances_[0])

            elif model_name == "LogisticRegression":
                print("Race importance (absolute coefficient):", model.coef_[0][0])
                histogram_data[dataset_name] = np.stack([y_proba, y_test_array, mask_target], axis=1)

        # Save and print the auc, precision, recall, and F1 for every classifier trained in each augmentation strategy
        auc_score = roc_auc_score(y_test_array, y_proba)
        precision = precision_score(y_test_array, y_pred, zero_division=0)
        recall = recall_score(y_test_array, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test_array, y_pred)
        f1 = f1_score(y_test_array, y_pred, zero_division=0)
        print(f"\n----- Classifier: {model_name} === AUC: {auc_score:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | Accuracy: {accuracy:.4f} | F1: {f1:.4f}")

        results.append({
            "Dataset": dataset_name,
            "Model": model_name,
            "Group": "Overall",
            "AUC": auc_score,
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "Accuracy": accuracy
        })

        # Now do the same for subgroups. Showcasing how the model impacts specifically the classifier performance on the target subgroup
        if classification_variable != label_1:
            mask_target = (df_test[label_2] == target_label_2_copy) & (df_test[label_1] == target_label_1)
        else:
            mask_target = df_test[label_2] == target_label_2_copy
        mask_non_target = ~mask_target

        for is_target, mask in [(True, mask_target), (False, mask_non_target)]:
            mask_name = f"{label_2} == {target_label_2_copy}" if is_target else f"{label_2} != {target_label_2_copy}"
            group_name = "Target" if is_target else "Non-Target"
            y_true_subset = y_test_array[mask]
            y_pred_subset = y_pred[mask]
            y_proba_subset = y_proba[mask]

            auc_subset = roc_auc_score(y_true_subset, y_proba_subset)
            precision_subset = precision_score(y_true_subset, y_pred_subset, zero_division=0)
            recall_subset = recall_score(y_true_subset, y_pred_subset, zero_division=0)
            accuracy_subset = accuracy_score(y_true_subset, y_pred_subset)
            f1_subset = f1_score(y_true_subset, y_pred_subset, zero_division=0)
            print(f"\n--- Subset: {mask_name} scores === AUC: {auc_subset:.4f} | Precision: {precision_subset:.4f} | Recall: {recall_subset:.4f} | Accuracy: {accuracy_subset:.4f} | F1: {f1_subset:.4f}")

            results.append({
                "Dataset": dataset_name,
                "Model": model_name,
                "Group": group_name,
                "AUC": auc_subset,
                "Precision": precision_subset,
                "Recall": recall_subset,
                "F1": f1_subset,
                "Accuracy": accuracy_subset
            })

In [ ]:
# Summarized classifier outputs used in results seciton.
results_df = pd.DataFrame(results)

group_names = ["Overall", "Target"]
metrics = ["AUC", "Recall", "Precision", "F1", "Accuracy"]

dataset_order = [
    "only_train",
    "train_combined",
    "train_plus_synthpop_id",
    "train_plus_id",
    "train_plus_synthpop_ood",
    "train_plus_ood",
    "train_plus_synthpop_both",
    "train_plus_both",
    "train_plus_real",
    "train_plus_ood_small",
    "train_plus_ood_large",
]

# The results are displayed for both the overall dataset and the target class (African American) only
for group in group_names:
    print(f"\n\n========== Group: {group} ==========\n")

    df_group = results_df[results_df["Group"] == group]

    formatted = []
    for model in df_group["Model"].unique():
        df_model = df_group[df_group["Model"] == model]
        df_model = df_model.set_index("Dataset").sort_index()
        for metric in metrics:
            row_label = (model, metric)
            row_values = df_model[metric].to_dict()
            formatted.append(pd.Series(row_values, name=row_label))

    result_table = pd.DataFrame(formatted).reindex(columns=dataset_order)
    result_table.index = pd.MultiIndex.from_tuples(result_table.index, names=["Model", "Metric"])

    display(result_table)

    # A summarized dataset is created, that computes the mean and std over the three classifiers for each strategy
    summary_df = df_group.groupby("Dataset")[metrics].agg(["mean", "std"])
    summary_formatted = pd.DataFrame(index=metrics, columns=dataset_order)

    for dataset in dataset_order:
        for metric in metrics:
            mean_val = summary_df.loc[dataset, (metric, "mean")]
            std_val = summary_df.loc[dataset, (metric, "std")]
            summary_formatted.loc[metric, dataset] = f"{mean_val:.3f}  ({std_val:.3f})"

    summary_formatted.index.name = "Metric"
    print(f"\n--- Summary (mean ± std across models) for group: {group} ---")
    display(summary_formatted)

In [ ]:
# Compute distance metrics for generated OOD samples of synthpop and CDTD-CFG to the removed subgroup
metrics_cdtd = compute_distance_metrics(removed_group_df, df_generated_OOD_filtered, cat_features, cont_features)
metrics_synthpop = compute_distance_metrics(removed_group_df, df_generated_synthpop_OOD, cat_features, cont_features)

print("CDTD-CFG OOD: ", metrics_cdtd)
print("Synthpop OOD: ", metrics_synthpop)

# Decode training data back to original feature space in order to compute distance metrics for the training data, and two feature specific subsets too
X_cat_train_decoded = cat_int_enc.inverse_transform(X_cat_train.numpy())
X_cont_train_real = cont_enc.inverse_transform(cont_std.inverse_transform(X_cont_train.numpy()))
df_train_raw = pd.DataFrame(np.concatenate([X_cat_train_decoded, X_cont_train_real], axis=1),
                            columns=cat_features + cont_features)
df_train_raw[cont_features] = df_train_raw[cont_features].apply(pd.to_numeric, errors="coerce")
df_train_raw[int_features] = df_train_raw[int_features].apply(pd.to_numeric, errors="coerce").astype(int)

df_train_filtered_label_1 = df_train_raw[
    (df_train_raw[label_1] == target_label_1)
].copy()
df_train_filtered_label_2 = df_train_raw[
    (df_train_raw[label_2] == target_label_2)
].copy()

metrics_train = compute_distance_metrics(removed_group_df, df_train_raw, cat_features, cont_features)
metrics_train_label_1 = compute_distance_metrics(removed_group_df, df_train_filtered_label_1, cat_features, cont_features)
metrics_train_label_2 = compute_distance_metrics(removed_group_df, df_train_filtered_label_2, cat_features, cont_features)
print("Train: ", metrics_train)
print("Train_label_1: ", metrics_train_label_1)
print("Train_label_2: ", metrics_train_label_2)

# -----------------------------------------------------------------------------------------------------------------------------------------------
# Plots histograms for showcasing the predicted probability of being classified as target = 1 for train only and train + OOD
bin_width_main = 0.0333333333333
bins_main = np.arange(0, 1 + bin_width_main, bin_width_main)

# An additional zoomed in histogram was created to show the distribution in the trainonly scenarios
bin_width_zoom = 0.000125
bins_zoom = np.arange(0, 0.005 + bin_width_zoom, bin_width_zoom)

for dataset_name, data in histogram_data.items():
    y_proba, y_true, mask = data[:, 0], data[:, 1], data[:, 2]

    # Create boolean masks to seperate between instances where the true target value = 1 and 0
    is_masked = (mask == 1)
    is_not_masked = (mask == 0)
    def get_group_data(mask_filter):
        return {
            'y0': y_proba[np.logical_and(mask_filter, y_true == 0)],
            'y1': y_proba[np.logical_and(mask_filter, y_true == 1)]
        }
    masked = get_group_data(is_masked)
    not_masked = get_group_data(is_not_masked)

    def plot_grouped_histogram(group_1, group_2, bins, title_suffix, dataset_name):
        counts_1_y0, _ = np.histogram(group_1['y0'], bins)
        counts_1_y1, _ = np.histogram(group_1['y1'], bins)
        counts_2_y0, _ = np.histogram(group_2['y0'], bins)
        counts_2_y1, _ = np.histogram(group_2['y1'], bins)

        bin_centers = bins[:-1] + (bins[1] - bins[0]) / 2
        bar_width = (bins[1] - bins[0]) / 3

        plt.figure(figsize=(10, 6))
        plt.bar(bin_centers - bar_width, counts_1_y0, width=bar_width, label='African American: not readmitted', alpha=0.7)
        plt.bar(bin_centers - bar_width, counts_1_y1, width=bar_width, bottom=counts_1_y0, label='African American: readmitted', alpha=0.7)

        plt.bar(bin_centers + bar_width, counts_2_y0, width=bar_width, label='Non African American: not readmitted', alpha=0.7)
        plt.bar(bin_centers + bar_width, counts_2_y1, width=bar_width, bottom=counts_2_y0, label='Non African American: readmitted', alpha=0.7)

        plt.title(f'Predicted Probability Histogram - {dataset_name} ({title_suffix})')
        plt.xlabel('Predicted Probability (y_proba)')
        plt.ylabel('Count')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    # Plot complete and zoomed-in histograms
    plot_grouped_histogram(masked, not_masked, bins_main, 'Full Range', dataset_name)
    if dataset_name == "only_train":
        plot_grouped_histogram(masked, not_masked, bins_zoom, 'Zoom: [0.0–0.005]', dataset_name)